In [ ]:
from dask.distributed import Client, progress
client = Client(processes=False)
client

Dask Arrays
===========
Dask arrays api allows to coordinate smaller _common_ arrays to represent a larger one.  
The chunks are distributed on each worker, splitting the work and memory usage.

![Dask array logo](https://examples.dask.org/_images/dask-array-black-text.svg)

Chunks can be composed of many ndarrays-like type (jax.numpy, sparse, pytorch tensor, Xarray, ...)  

The dask array can be used almost identically as a numpy array.





In [ ]:
import dask.array as da
x = da.random.random((400, 400), chunks=(200, 200))
x

In [ ]:
y = x[::2, :100]
y

In [ ]:
z = y.sum(axis=0)
z

In [ ]:
z.compute()

In [ ]:
da.std(x)

In [ ]:
da.std(x).compute()

In [ ]:
da.std(x).visualize()

Supported operations
--------------------

- Arithmetic and scalar mathematics: +, *, exp, log, ...
- Reductions along axes: sum(), mean(), std(), sum(axis=0), ...
- Tensor contractions / dot products / matrix multiply / tensordot / einsum...
- Axis reordering / transpose: transpose
- Slicing: x[:100, 500:100:-2]
- Some linear algebra

[List of supported operations](https://docs.dask.org/en/stable/array-numpy-compatibility.html)

In [ ]:
M = da.random.random((6000, 6000), chunks=(1000, 1000))
y = da.random.random((6000, 1), chunks=(1000, 1))
My = da.expm1(M) @ y
My

In [ ]:
My.visualize()

In [ ]:
My.compute()

In [ ]:
da.einsum("ij,jk->ik", M, y).visualize()

Creating, reading and saving dask arrays
----------------------------------------

We can create dask arrays from normal arrays.

In [ ]:
import numpy as np
da.from_array(np.ones((1000, 1000)), chunks=(100, 100))

`from_array` can create array from many array like packages.

In [ ]:
import sparse
N = 1000
sp = sparse.COO(
    (np.arange(N-1), np.arange(1, N)), np.arange(-N//2, N//2-1), 
    shape=(N, N)
)
sp

In [ ]:
da.from_array(sp, chunks=(100, 100))

This is easy, but at creation, the full array must be in memory on the head worker.

As easy but more efficient: using dask functions!
- ones
- zeros
- arange
- random
- linspace
- ...

In [ ]:
da.ones((1000, 1000), chunks=(100, 100))

This is more efficient, but quite limited.

A method to be both efficient and versatile is to use `delayed`:

In [ ]:
from dask import delayed
N = 100
@delayed
def make_block(N, row, col):
    return np.ones((N, N)) * np.exp(-(row - col)**2)

col = []
for i in range(10):
    row = []
    for j in range(10):
        row.append(
            da.from_delayed(
                make_block(N, i, j),
                shape=(N, N),
                dtype=np.float64,
            )
        )
    col.append(da.concatenate(row, axis=0))
mat = da.concatenate(col, axis=1)
mat

In [ ]:
mat.visualize()

There are also methods to save / load dask array in chunks.  
`dask` has functions to support numpy format (.npy), hdf5 and object storage (zarr):
- `from_npy_stack`, `to_npy_stack`
- `from_zarr`, `to_zarr`
- `from_tiledb`, `to_tiledb`
- `to_hdf5`

In [ ]:
da.to_npy_stack("save_rnd", da.random.random((3000, 3000), chunks=(1000, 1000)), axis=0)
random_array = da.from_npy_stack("save_rnd")
random_array

Exercise: Numpy buzy work
-------------------------
Here is some numpy workload, let's rewrite it with dask.

In [ ]:
N = 1000
mat1 = np.eye(N)
mat2 = np.linspace(0, 777, N) % 1
mat2 = np.multiply.outer(mat2, mat2)
mat1[mat2 > 0.5] = -0.1
mat2 = np.tan(mat2)
mat3 = mat1 * mat2
print(np.max(mat3))

### Solution
<!---
N = 1000
mat1 = da.eye(N)
mat2 = da.linspace(0, 777, N) % 1
mat2 = da.multiply.outer(mat2, mat2)
mat1[mat2 > 0.5] = -0.1
mat2 = da.tan(mat2)
mat3 = mat1 * mat2
print(da.max(mat3).compute())
--->

## Exercise: ODE

Let's use dask to integrate a differential equation: $dv / dt = f(v)$.

We have a working numpy version which work well for small ``N``.  
We want to use dask to solve a similar but larger system.

In [ ]:
N = 10000

def norm(v):
    """
    Norm of the vector
    """
    return (v * v.conj()).sum()**0.5


H = np.random.rand(N, N) + np.random.rand(N, N) *1j
H = 1j * (H + H.transpose().conj()) / 100


def f(v):
    """
    Derivative function
    """
    return H @ v


def rk4(initial, f, dt):
    """
    Simple fixed step lenght runke-kutta integration.
    """
    d0 = f(v)
    d1 = f(v + d0 * 0.5 * dt)
    d2 = f(v + d1 * 0.5 * dt)
    d3 = f(v + d2 * dt)
    return v + (d0 / 6 + d1 / 3 + d2 / 3 + d3 / 6) * dt


def integrate(v, f, dt, N_step):
    """
    Do N_step of runge kutta integration.
    """
    for _ in range(N_step):
        v = rk4(v, f, dt)
    return v


v = np.random.rand(N, 1)
v = v / norm(v)

%time v = integrate(v, f, 0.001, 100)

#Sanity check
print(norm(v))

### Solution
<!---
N = 10000

def norm(v):
    """
    Norm of the vector
    """
    return (v * v.conj()).sum()**0.5


H = (
    da.random.random((N, N), chunks=(N//2, N//2))
    + 1j * da.random.random((N, N), chunks=(N//2, N//2))
)
H = 1j * (H + H.transpose().conj()) / 100


def f(v):
    """
    Derivative function
    """
    return H @ v


def rk4(initial, f, dt):
    """
    Simple fixed step lenght runke-kutta integration.
    """
    d0 = f(v)
    d1 = f(v + d0 * 0.5 * dt)
    d2 = f(v + d1 * 0.5 * dt)
    d3 = f(v + d2 * dt)
    return v + (d0 / 6 + d1 / 3 + d2 / 3 + d3 / 6) * dt


def integrate(v, f, dt, N_step):
    """
    Do N_step of runge kutta integration.
    """
    for _ in range(N_step):
        v = rk4(v, f, dt)
    return v


v = da.random.random((N, 1), chunks=(N//2, 1))
v = v / norm(v)

%time v = integrate(v, f, 0.001, 100)

#Sanity check
%time norm(v).compute()

--->